In [ ]:
%pip install vega_datasets seaborn altair anywidget==0.9.13

# Bringing Notebooks to Life with anywidget

This notebook contains the `anywidget` demo from the
[2024 ACM CHI Workshop on Human-Notebook Interactions](https://humannotebookinteractions.github.io/).

You can watch the [video](https://youtu.be/Uzm9_2ZBfxo) and follow along, or just run the cells on your own. To learn more, check out the [docs](https://anywidget.dev/en/getting-started/) or the Jupyter [blog post](https://blog.jupyter.org/anywidget-jupyter-widgets-made-easy-164eb2eae102).

## Status Quo: static outputs

In [ ]:
from vega_datasets import data

df = data.cars()
df.head()

In [ ]:
import seaborn as sns

sns.scatterplot(x="Horsepower", y="Miles_per_Gallon", hue="Origin", data=df)

## One-way data flow (surface level integration)

In [ ]:
import altair as alt

brush = alt.selection_interval()

points = alt.Chart(df).mark_point().encode(
    x="Horsepower",
    y="Miles_per_Gallon",
    color=alt.condition(brush, "Origin", alt.value("lightgray")),
    tooltip=["Horsepower", "Miles_per_Gallon"],
).add_params(
    brush
)

bars = alt.Chart(df).mark_bar().encode(
    y="Origin",
    color="Origin",
    x="count(Origin)"
).transform_filter(
    brush
)

chart = (points & bars)
chart

In [ ]:
chart.selection # <- How can we access the JavaScript selection

## Two-way data flow (with anywidget)

In [ ]:
import anywidget
import traitlets

class ChartWidget(anywidget.AnyWidget):
    spec = traitlets.Dict().tag(sync=True)
    selection = traitlets.Dict(sync=True)

    _esm = """
    import embed from "https://cdn.jsdelivr.net/npm/vega-embed@6/+esm";
    async function render({ model, el }) {
        let spec = model.get("spec");
        let api = await embed(el, spec);
        api.view.addSignalListener(spec.params[0].name, (_, update) => {
            model.set("selection", update);
            model.save_changes();
        })
    }
    export default { render };
    """

widget = ChartWidget(spec=chart.to_dict())
widget

In [ ]:
# Make a new selection and re-run this cell! Notice output changes
widget.selection

## Composing widgets together

In [ ]:
import ipywidgets

output = ipywidgets.Output()

@output.capture(clear_output=True)
def on_change(change):
    sub = df
    selection = change.new
    for field, (lower, upper) in selection.items():
        sub = sub[(sub[field] > lower) & (sub[field] < upper)]
    display(sub)

widget.observe(on_change, names="selection")
ipywidgets.VBox([widget, output])